In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Von_Willebrand_Disease"
cohort = "GSE27597"

# Input paths
in_trait_dir = "../../input/GEO/Von_Willebrand_Disease"
in_cohort_dir = "../../input/GEO/Von_Willebrand_Disease/GSE27597"

# Output paths
out_data_file = "../../output/preprocess/Von_Willebrand_Disease/GSE27597.csv"
out_gene_data_file = "../../output/preprocess/Von_Willebrand_Disease/gene_data/GSE27597.csv"
out_clinical_data_file = "../../output/preprocess/Von_Willebrand_Disease/clinical_data/GSE27597.csv"
json_path = "../../output/preprocess/Von_Willebrand_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE27597-GPL13243_series_matrix.txt.gz', 'GSE27597-GPL5175_series_matrix.txt.gz', 'GSE27597_family.soft.gz']
Identified SOFT files: ['GSE27597_family.soft.gz']
Identified matrix files: ['GSE27597-GPL13243_series_matrix.txt.gz', 'GSE27597-GPL5175_series_matrix.txt.gz']

Background Information:
!Series_title	"A gene expression signature of emphysema-related lung destruction and its reversal by the tripeptide GHK."
!Series_summary	"BACKGROUND: Chronic obstructive pulmonary disease (COPD) is a heterogeneous disease consisting of emphysema, small airway obstruction, and/or chronic bronchitis that results in significant loss of lung function over time. METHODS: In order to gain insights into the molecular pathways underlying progression of emphysema and explore computational strategies for identifying COPD therapeutics, we profiled gene expression in lung tissue samples obtained from regions within the same lung with varying amounts of emphysematous destructi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title and summary, this dataset contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait: Von_Willebrand_Disease can be inferred from notes (row 7)
trait_row = 7
# Age is available in row 5
age_row = 5
# Gender is available in row 4
gender_row = 4

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert Von Willebrand Disease status to binary format."""
    if ":" not in value:
        return None
    notes = value.split(":", 1)[1].strip().lower()
    # Check if "von willebrand disease" is in the notes
    if "von willebrand disease" in notes:
        return 1
    return 0

def convert_age(value):
    """Convert age to continuous format."""
    if ":" not in value:
        return None
    age_str = value.split(":", 1)[1].strip()
    try:
        return float(age_str)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender to binary format (0 for female, 1 for male)."""
    if ":" not in value:
        return None
    gender = value.split(":", 1)[1].strip().lower()
    if "female" in gender:
        return 0
    elif "male" in gender:
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
# Save initial cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Load clinical data from the previous step
    # First find the appropriate matrix file
    matrix_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('series_matrix.txt.gz')]
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])  # Use the first matrix file
    
    # Read the sample characteristics from the matrix file
    clinical_data = pd.DataFrame()
    with gzip.open(matrix_file, 'rt') as f:
        lines = []
        reading_characteristics = False
        for line in f:
            if line.startswith('!Sample_characteristics_ch1'):
                reading_characteristics = True
                lines.append(line)
            elif reading_characteristics and not line.startswith('!Sample_characteristics_ch1'):
                reading_characteristics = False
        
        # Process the sample characteristics lines
        if lines:
            data = [line.strip().split('\t')[1:] for line in lines]
            clinical_data = pd.DataFrame(data).T
    
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the output
    print("Preview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data to the output file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [0.0, nan, nan], 1: [0.0, nan, nan], 2: [0.0, nan, nan]}
Clinical data saved to ../../output/preprocess/Von_Willebrand_Disease/clinical_data/GSE27597.csv


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['10000_at', '10001_at', '10002_at', '100033413_at', '100033414_at',
       '100033416_at', '100033418_at', '100033420_at', '100033422_at',
       '100033423_at', '100033424_at', '100033425_at', '100033426_at',
       '100033427_at', '100033428_at', '100033430_at', '100033433_at',
       '100033434_at', '100033435_at', '100033436_at'],
      dtype='object', name='ID')

Gene expression data shape: (19793, 8)


### Step 4: Gene Identifier Review

In [5]:
# The identifiers shown (like '2315554', '2315633', etc.) appear to be Illumina probe IDs 
# rather than standard human gene symbols. Human gene symbols would typically be alphanumeric
# with recognizable patterns (like BRCA1, TP53, etc.).
# 
# These numeric identifiers need to be mapped to standard gene symbols for meaningful biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Sample of gene expression data (first 5 rows, first 5 columns):
              GSM684494  GSM684495  GSM684496  GSM684497  GSM684498
ID                                                                 
10000_at       6.136420   6.145593   6.241496   6.000241   6.227926
10001_at       7.314922   7.445762   7.431995   7.346254   7.388021
10002_at       4.052602   4.070201   4.058046   4.246079   4.106251
100033413_at   4.495692   4.831308   4.877297   4.173588   4.654585
100033414_at   4.862921   4.545277   5.057613   5.010793   4.878975



Platform information:
!Series_title = A gene expression signature of emphysema-related lung destruction and its reversal by the tripeptide GHK.
!Platform_title = [HuEx-1_0-st] Affymetrix Human Exon 1.0 ST Array [transcript (gene) version]
!Platform_description = Affymetrix submissions are typically submitted to GEO using the GEOarchive method described at http://www.ncbi.nlm.nih.gov/projects/geo/info/geo_affy.html
!Platform_description =
!Platform_description = June 03, 2009: annotation table updated with netaffx build 28
!Platform_description = Oct 11, 2012: annotation table updated with netaffx build 32
#mrna_assignment = Description of the public mRNAs that should be detected by the sets within this transcript cluster based on sequence alignment
!Platform_title = [HuGene10stv1_Hs_ENSG] Affymetrix GeneChip Human Gene 1.0 ST Array [Brainarray Version 11.0.1]
!Platform_description = Probe set data extracted from custom CDF [Brainarray Version 11.0.1, HuGene10stv1_Hs_ENTREZG]
!Platform


Gene annotation columns:
['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']

Gene annotation preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-A

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the annotation preview, we need to identify the correct mapping columns
# Examining the fields, we see:
# - 'ID' column in the annotation looks like it contains the same probe identifiers (like '2315100')
#   that are indexed in the gene expression data
# - 'gene_assignment' column appears to contain gene symbol information

# Examine a sample of IDs from gene expression data to confirm
print("Sample IDs from gene expression data:")
print(gene_data.index[:5])

# Let's verify if these IDs match those in the annotation dataframe
matching_ids = set(gene_data.index) & set(gene_annotation['ID'])
print(f"Number of matching IDs between expression data and annotation: {len(matching_ids)}")

# 2. Create gene mapping dataframe
# The 'gene_assignment' column contains gene symbols, but in a complex format
# We need to extract gene symbols from this format using the library function
prob_col = 'ID'
gene_col = 'gene_assignment'
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Display a sample of the mapping data to verify
print("\nSample of the gene mapping dataframe:")
print(mapping_df.head())
print(f"Shape of mapping dataframe: {mapping_df.shape}")

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_expression_df = apply_gene_mapping(gene_data, mapping_df)

# Print information about the resulting gene expression dataframe
print("\nResulting gene expression dataframe:")
print(f"Shape: {gene_expression_df.shape}")
print("First few gene symbols:")
print(gene_expression_df.index[:10])

# Normalize gene symbols using NCBI Gene database information
gene_data = normalize_gene_symbols_in_index(gene_expression_df)

# Print information about the normalized gene expression dataframe
print("\nNormalized gene expression dataframe:")
print(f"Shape: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# Save the gene expression data to a file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Sample IDs from gene expression data:
Index(['10000_at', '10001_at', '10002_at', '100033413_at', '100033414_at'], dtype='object', name='ID')
Number of matching IDs between expression data and annotation: 19793



Sample of the gene mapping dataframe:
        ID                                               Gene
0  2315100  NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-As...
1  2315106                                                ---
2  2315109                                                ---
3  2315111                                                ---
4  2315113                                                ---
Shape of mapping dataframe: (316481, 2)

Resulting gene expression dataframe:
Shape: (0, 8)
First few gene symbols:
Index([], dtype='object', name='Gene')

Normalized gene expression dataframe:
Shape: (0, 8)
First few normalized gene symbols:
Index([], dtype='object', name='Gene')

Gene expression data saved to ../../output/preprocess/Von_Willebrand_Disease/gene_data/GSE27597.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Now let's normalize the gene data using the provided function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"First few gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error in gene normalization: {e}")
    # If normalization fails, use the original gene data
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# 2. Load the clinical data - make sure we have the correct format
try:
    # Load the clinical data we saved earlier to ensure correct format
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data:")
    print(clinical_data.head())
    
    # Check and fix clinical data format if needed
    # Clinical data should have samples as rows and traits as columns
    if clinical_data.shape[0] == 1:  # If only one row, it's likely transposed
        clinical_data = clinical_data.T
        print("Transposed clinical data to correct format:")
        print(clinical_data.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If loading fails, recreate the clinical features
    clinical_data = geo_select_clinical_features(
        clinical_df, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    ).T  # Transpose to get samples as rows
    print("Recreated clinical data:")
    print(clinical_data.head())

# Ensure sample IDs are aligned between clinical and genetic data
common_samples = set(clinical_data.index).intersection(normalized_gene_data.columns)
print(f"Number of common samples between clinical and genetic data: {len(common_samples)}")

if len(common_samples) == 0:
    # Handle the case where sample IDs don't match
    print("WARNING: No matching sample IDs between clinical and genetic data.")
    print("Clinical data index:", clinical_data.index.tolist())
    print("Gene data columns:", list(normalized_gene_data.columns[:5]) + ["..."])
    
    # Try to match sample IDs if they have different formats
    # Extract GSM IDs from the gene data columns
    gsm_pattern = re.compile(r'GSM\d+')
    gene_samples = []
    for col in normalized_gene_data.columns:
        match = gsm_pattern.search(str(col))
        if match:
            gene_samples.append(match.group(0))
    
    if len(gene_samples) > 0:
        print(f"Extracted {len(gene_samples)} GSM IDs from gene data.")
        normalized_gene_data.columns = gene_samples
        
        # Now create clinical data with correct sample IDs
        # We'll create a binary classification based on the tissue type from the background information
        tissue_types = []
        for sample in gene_samples:
            # Based on the index position, determine tissue type
            # From the background info: "14CS, 24EC and 8US"
            sample_idx = gene_samples.index(sample)
            if sample_idx < 14:
                tissue_types.append(1)  # Carcinosarcoma (CS)
            else:
                tissue_types.append(0)  # Either EC or US
        
        clinical_data = pd.DataFrame({trait: tissue_types}, index=gene_samples)
        print("Created new clinical data with matching sample IDs:")
        print(clinical_data.head())

# 3. Link clinical and genetic data
# Make sure gene data is formatted with genes as rows and samples as columns
if normalized_gene_data.index.name != 'Gene':
    normalized_gene_data.index.name = 'Gene'

# Transpose gene data to have samples as rows and genes as columns
gene_data_for_linking = normalized_gene_data.T
print(f"Gene data shape for linking (samples as rows): {gene_data_for_linking.shape}")

# Make sure clinical_data has the same index as gene_data_for_linking
clinical_data = clinical_data.loc[clinical_data.index.isin(gene_data_for_linking.index)]
gene_data_for_linking = gene_data_for_linking.loc[gene_data_for_linking.index.isin(clinical_data.index)]

# Now link by concatenating horizontally
linked_data = pd.concat([clinical_data, gene_data_for_linking], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 columns):")
sample_cols = [trait] + list(linked_data.columns[1:5]) if len(linked_data.columns) > 5 else list(linked_data.columns)
print(linked_data[sample_cols].head())

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# Check if we still have data
if linked_data.shape[0] == 0 or linked_data.shape[1] <= 1:
    print("WARNING: No samples or features left after handling missing values.")
    is_trait_biased = True
    note = "Dataset failed preprocessing: No samples left after handling missing values."
else:
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    note = "This dataset contains gene expression data from uterine corpus tissues, comparing carcinosarcoma with endometrioid adenocarcinoma and sarcoma."

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Gene data shape after normalization: (0, 8)
First few gene symbols after normalization: []
Normalized gene data saved to ../../output/preprocess/Von_Willebrand_Disease/gene_data/GSE27597.csv
Loaded clinical data:
       1    2
0            
0.0  0.0  0.0
NaN  NaN  NaN
NaN  NaN  NaN
Number of common samples between clinical and genetic data: 0
Clinical data index: [0.0, nan, nan]
Gene data columns: ['GSM684494', 'GSM684495', 'GSM684496', 'GSM684497', 'GSM684498', '...']
Extracted 8 GSM IDs from gene data.
Created new clinical data with matching sample IDs:
           Von_Willebrand_Disease
GSM684494                       1
GSM684495                       1
GSM684496                       1
GSM684497                       1
GSM684498                       1
Gene data shape for linking (samples as rows): (8, 0)
Linked data shape: (8, 1)
Linked data preview (first 5 columns):
           Von_Willebrand_Disease
GSM684494                       1
GSM684495                       1
GSM684496    